# Soccer - PPO

---


### 1. Start the Environment

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
import gc
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
from unityagents import UnityEnvironment

env = UnityEnvironment(file_name="Soccer.app", seed=1)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 2
        Number of External Brains : 2
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: GoalieBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Unity brain name: StrikerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 6
        Vector Action descriptions: , , , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we obtain separate brains for the striker and goalie agents.

In [3]:
# print the brain names
print(env.brain_names)

# set the goalie brain
g_brain_name = env.brain_names[0]
g_brain = env.brains[g_brain_name]

# set the striker brain
s_brain_name = env.brain_names[1]
s_brain = env.brains[s_brain_name]

['GoalieBrain', 'StrikerBrain']


### 2. Examine the State and Action Spaces

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)

# number of agents 
num_g_agents = len(env_info[g_brain_name].agents)
print('Number of goalie agents:', num_g_agents)
num_s_agents = len(env_info[s_brain_name].agents)
print('Number of striker agents:', num_s_agents)

# number of actions
g_action_size = g_brain.vector_action_space_size
print('Number of goalie actions:', g_action_size)
s_action_size = s_brain.vector_action_space_size
print('Number of striker actions:', s_action_size)

# examine the state space 
g_states = env_info[g_brain_name].vector_observations
g_state_size = g_states.shape[1]
print('There are {} goalie agents. Each receives a state with length: {}'.format(g_states.shape[0], g_state_size))
s_states = env_info[s_brain_name].vector_observations
s_state_size = s_states.shape[1]
print('There are {} striker agents. Each receives a state with length: {}'.format(s_states.shape[0], s_state_size))

Number of goalie agents: 2
Number of striker agents: 2
Number of goalie actions: 4
Number of striker actions: 6
There are 2 goalie agents. Each receives a state with length: 336
There are 2 striker agents. Each receives a state with length: 336


### Policy

In [5]:
class Policy(nn.Module):

    def __init__(self, state_size, action_size, hidden_0, hidden_1, hidden_2):
        super(Policy, self).__init__()
               
        self.fc1 = nn.Linear(state_size*4, hidden_0)
        self.fc2 = nn.Linear(hidden_0, hidden_1)
#        self.fc3 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_1, action_size)
        
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#        x = F.relu(self.fc3(x))
        return F.softmax(self.fc3(x), dim=1)

### Policy Improvement

In [6]:
def learn(epochs, agent, optimizer, prob_list, state_list, action_list, reward_list, discount, epsilon, beta):
    for _ in range(epochs):
        L = -clipped_surrogate(agent, prob_list, state_list, action_list, reward_list, discount, epsilon, beta)
        optimizer.zero_grad()
        L.backward()
#        nn.utils.clip_grad_norm_(agent.parameters(), 1)
        optimizer.step()
        del [L]
        gc.collect()
        
def clipped_surrogate(agent, old_probs, states, actions, rewards, discount, epsilon, beta):

    # get discounted future rewards at each step
    discount = discount**np.arange(len(rewards))
    rewards = rewards * discount
    rewards_future = rewards[::-1].cumsum(axis=0)[::-1]

    # normalize rewards
    mean = np.mean(rewards_future)
    std = np.std(rewards_future) + 1.0e-10
    rewards_normalized = (rewards_future - mean)/std

    # convert everything into pytorch tensors and move to gpu if available
    actions = torch.tensor(actions, dtype=torch.int8, device=device)
    old_probs = torch.tensor(old_probs, dtype=torch.float, device=device)
    rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)

    # get new probability of each action given the state and latest policy
    states = torch.stack(states)
    states = states.view(-1, g_state_size*4)
    all_probs = agent(states)
    new_probs = []
    for i in range(len(rewards)):
        new_probs.append(all_probs[i][actions[i]])
    new_probs = torch.tensor(new_probs, dtype=torch.float, device=device)
    new_probs = Variable(new_probs, requires_grad = True)
    # ratio for clipping - how much better is new action choice over old choice
    ratio = new_probs/old_probs

    # clipped function - don't update weights too much when finds much better choice
    clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)
    clipped_surrogate = torch.min(ratio*rewards, clip*rewards)

    # regularization term to steer new_policy towards equal probability of actions
    # add 1.e-10 to avoid log(0) which gives nan
    entropy = -(new_probs*torch.log(old_probs+1.e-10)+ \
        (1.0-new_probs)*torch.log(1.0-old_probs+1.e-10))
    
    # this returns an average of all the entries of the tensor
    # effective computing L_sur^clip / T
    # averaged over time-step and number of trajectories
    # this is desirable because we have normalized our rewards
    return torch.mean(clipped_surrogate + beta*entropy)

# combine all 4 players' state spaces together
def combine_states(g_states, s_states):
    s_states = torch.from_numpy(s_states).float().to(device)
    g_states = torch.from_numpy(g_states).float().to(device)
    states_all = torch.cat((s_states, g_states), dim=1)
    states_all = states_all.view(-1, g_state_size*4)
    return states_all

def act(agent, state, action_size):
    prob = agent(state).squeeze().cpu().detach().numpy()
    action = np.random.choice(action_size, p=prob)
    prob = prob[action]
    return prob, action


### Train Agents

In [7]:
episodes = 10000
learning_rate = 5e-4
discount= 0.995
t_max = 600

hidden_0 = 512
hidden_1 = 256
hidden_2 = 0

epsilon = .2
eps_decay = .999
beta = .03
beta_decay = .999
epochs = 10

In [ ]:
striker_0 = Policy(s_state_size, s_action_size, hidden_0, hidden_1, hidden_2).to(device)
goalie_0 = Policy(g_state_size, g_action_size, hidden_0, hidden_1, hidden_2).to(device)
striker_0_optimizer = optim.Adam(striker_0.parameters(), lr=learning_rate)
goalie_0_optimizer = optim.Adam(goalie_0.parameters(), lr=learning_rate)

team_0_score = 0
team_1_score = 0
team_0_delta = 0
team_0_lead = []

for e in range(episodes):                  
    
    state_list = []
    s_0_reward_list = []
    s_0_prob_list = []
    s_0_action_list = []
    g_0_reward_list = []
    g_0_prob_list = []
    g_0_action_list = []

    env_info = env.reset(train_mode=True)                     
    g_states = env_info[g_brain_name].vector_observations  
    s_states = env_info[s_brain_name].vector_observations  

    g_scores = np.zeros(num_g_agents)                      
    s_scores = np.zeros(num_s_agents)           

    for i in range(t_max):
        
        # combine all 4 agents' state spaces together
        states_all = combine_states(g_states, s_states)
        
        # select actions and send to environment
        g0_prob, g0_action = act(goalie_0, states_all, g_action_size)
        s0_prob, s0_action = act(striker_0, states_all, s_action_size)
        g_actions = np.array([g0_action, np.random.choice(g_action_size)])
        s_actions = np.array([s0_action, np.random.choice(s_action_size)])
        actions = dict(zip([g_brain_name, s_brain_name], [g_actions, s_actions]))
        env_info = env.step(actions)              

        # get next states
        g_next_states = env_info[g_brain_name].vector_observations         
        s_next_states = env_info[s_brain_name].vector_observations

        # get reward and store experience (s,a,r,p) in lists for striker and goalie
        g_rewards = env_info[g_brain_name].rewards  
        s_rewards = env_info[s_brain_name].rewards
        state_list.append(states_all)
        
        s_0_reward_list.append(np.array(s_rewards[0]))
        s_0_prob_list.append(s0_prob)
        s_0_action_list.append(s0_action)
        g_0_reward_list.append(np.array(g_rewards[0]))
        g_0_prob_list.append(g0_prob)
        g_0_action_list.append(g0_action)
        
        #update scores
        g_scores += g_rewards
        s_scores += s_rewards
        
        # check if episode finished
        done = np.any(env_info[g_brain_name].local_done)  

        # exit loop if episode finished
        if done:                                           
            break

        # roll over states to next time step
        g_states = g_next_states
        s_states = s_next_states

    # calculate scores and running team lead
    team_0_score = round(g_scores[0] + s_scores[0], 2)
    team_1_score = round(g_scores[1] + s_scores[1], 2)
    team_0_delta = round(team_0_score + team_0_delta, 2)
    team_0_lead.append(team_0_delta)
                
    # agents learn
    learn(epochs, striker_0, striker_0_optimizer, s_0_prob_list, state_list, s_0_action_list, s_0_reward_list, discount, epsilon, beta)
    learn(epochs, goalie_0, goalie_0_optimizer, g_0_prob_list, state_list, g_0_action_list, g_0_reward_list, discount, epsilon, beta)

    # reduce clipping and regularization (exploration) over time
    epsilon *= eps_decay                
    beta *= beta_decay
    
    print('Episode {}:\tSteps: {}\tTeam 1 Lead: {}\tTeam 1: {}\tTeam 2: {}'.format(
                        e+1, i+1, team_0_delta, team_0_score, team_1_score))

    if (e % 10 == 0) & (e != 0):
        torch.save(striker_0.state_dict(), 'checkpoint_striker_0.pth')
        torch.save(goalie_0.state_dict(), 'checkpoint_goalie_0.pth')
        

Episode 1:	Steps: 550	Team 1 Lead: -1.1	Team 1: -1.1	Team 2: 1.1
Episode 2:	Steps: 600	Team 1 Lead: -1.1	Team 1: 0.0	Team 2: 0.0
Episode 3:	Steps: 472	Team 1 Lead: -2.2	Team 1: -1.1	Team 2: 1.1
Episode 4:	Steps: 375	Team 1 Lead: -1.1	Team 1: 1.1	Team 2: -1.1
Episode 5:	Steps: 557	Team 1 Lead: -2.2	Team 1: -1.1	Team 2: 1.1
Episode 6:	Steps: 36	Team 1 Lead: -3.3	Team 1: -1.1	Team 2: 1.1
Episode 7:	Steps: 600	Team 1 Lead: -3.3	Team 1: 0.0	Team 2: 0.0
Episode 8:	Steps: 600	Team 1 Lead: -3.3	Team 1: 0.0	Team 2: 0.0
Episode 9:	Steps: 593	Team 1 Lead: -4.4	Team 1: -1.1	Team 2: 1.1
Episode 10:	Steps: 600	Team 1 Lead: -4.4	Team 1: 0.0	Team 2: 0.0
Episode 11:	Steps: 600	Team 1 Lead: -4.4	Team 1: 0.0	Team 2: 0.0
Episode 12:	Steps: 391	Team 1 Lead: -3.3	Team 1: 1.1	Team 2: -1.1
Episode 13:	Steps: 495	Team 1 Lead: -2.2	Team 1: 1.1	Team 2: -1.1
Episode 14:	Steps: 200	Team 1 Lead: -1.1	Team 1: 1.1	Team 2: -1.1
Episode 15:	Steps: 600	Team 1 Lead: -1.1	Team 1: 0.0	Team 2: 0.0
Episode 16:	Steps: 427	Tea

Episode 128:	Steps: 109	Team 1 Lead: 0.0	Team 1: -1.1	Team 2: 1.1
Episode 129:	Steps: 222	Team 1 Lead: -1.1	Team 1: -1.1	Team 2: 1.1
Episode 130:	Steps: 466	Team 1 Lead: 0.0	Team 1: 1.1	Team 2: -1.1
Episode 131:	Steps: 328	Team 1 Lead: 1.1	Team 1: 1.1	Team 2: -1.1
Episode 132:	Steps: 600	Team 1 Lead: 1.1	Team 1: 0.0	Team 2: 0.0
Episode 133:	Steps: 42	Team 1 Lead: 0.0	Team 1: -1.1	Team 2: 1.1
Episode 134:	Steps: 53	Team 1 Lead: 1.1	Team 1: 1.1	Team 2: -1.1
Episode 135:	Steps: 600	Team 1 Lead: 1.1	Team 1: 0.0	Team 2: 0.0
Episode 136:	Steps: 501	Team 1 Lead: 2.2	Team 1: 1.1	Team 2: -1.1
Episode 137:	Steps: 600	Team 1 Lead: 2.2	Team 1: 0.0	Team 2: 0.0
Episode 138:	Steps: 88	Team 1 Lead: 3.3	Team 1: 1.1	Team 2: -1.1
Episode 139:	Steps: 600	Team 1 Lead: 3.3	Team 1: 0.0	Team 2: 0.0
Episode 140:	Steps: 121	Team 1 Lead: 4.4	Team 1: 1.1	Team 2: -1.1
Episode 141:	Steps: 88	Team 1 Lead: 5.5	Team 1: 1.1	Team 2: -1.1
Episode 142:	Steps: 94	Team 1 Lead: 6.6	Team 1: 1.1	Team 2: -1.1
Episode 143:	Steps

Episode 253:	Steps: 600	Team 1 Lead: 8.8	Team 1: 0.0	Team 2: 0.0
Episode 254:	Steps: 170	Team 1 Lead: 9.9	Team 1: 1.1	Team 2: -1.1
Episode 255:	Steps: 187	Team 1 Lead: 8.8	Team 1: -1.1	Team 2: 1.1
Episode 256:	Steps: 600	Team 1 Lead: 8.8	Team 1: 0.0	Team 2: 0.0
Episode 257:	Steps: 600	Team 1 Lead: 8.8	Team 1: 0.0	Team 2: 0.0
Episode 258:	Steps: 378	Team 1 Lead: 9.9	Team 1: 1.1	Team 2: -1.1
Episode 259:	Steps: 122	Team 1 Lead: 8.8	Team 1: -1.1	Team 2: 1.1
Episode 260:	Steps: 56	Team 1 Lead: 9.9	Team 1: 1.1	Team 2: -1.1
Episode 261:	Steps: 379	Team 1 Lead: 8.8	Team 1: -1.1	Team 2: 1.1
Episode 262:	Steps: 486	Team 1 Lead: 9.9	Team 1: 1.1	Team 2: -1.1
Episode 263:	Steps: 102	Team 1 Lead: 8.8	Team 1: -1.1	Team 2: 1.1
Episode 264:	Steps: 144	Team 1 Lead: 7.7	Team 1: -1.1	Team 2: 1.1
Episode 265:	Steps: 247	Team 1 Lead: 6.6	Team 1: -1.1	Team 2: 1.1
Episode 266:	Steps: 600	Team 1 Lead: 6.6	Team 1: 0.0	Team 2: 0.0
Episode 267:	Steps: 336	Team 1 Lead: 7.7	Team 1: 1.1	Team 2: -1.1
Episode 268:	St

Episode 377:	Steps: 600	Team 1 Lead: 16.5	Team 1: 0.0	Team 2: 0.0
Episode 378:	Steps: 600	Team 1 Lead: 16.5	Team 1: 0.0	Team 2: 0.0
Episode 379:	Steps: 95	Team 1 Lead: 15.4	Team 1: -1.1	Team 2: 1.1
Episode 380:	Steps: 600	Team 1 Lead: 15.4	Team 1: 0.0	Team 2: 0.0
Episode 381:	Steps: 525	Team 1 Lead: 14.3	Team 1: -1.1	Team 2: 1.1
Episode 382:	Steps: 469	Team 1 Lead: 13.2	Team 1: -1.1	Team 2: 1.1
Episode 383:	Steps: 600	Team 1 Lead: 13.2	Team 1: 0.0	Team 2: 0.0
Episode 384:	Steps: 211	Team 1 Lead: 12.1	Team 1: -1.1	Team 2: 1.1
Episode 385:	Steps: 124	Team 1 Lead: 11.0	Team 1: -1.1	Team 2: 1.1
Episode 386:	Steps: 600	Team 1 Lead: 11.0	Team 1: 0.0	Team 2: 0.0
Episode 387:	Steps: 114	Team 1 Lead: 12.1	Team 1: 1.1	Team 2: -1.1
Episode 388:	Steps: 600	Team 1 Lead: 12.1	Team 1: 0.0	Team 2: 0.0
Episode 389:	Steps: 176	Team 1 Lead: 13.2	Team 1: 1.1	Team 2: -1.1
Episode 390:	Steps: 123	Team 1 Lead: 14.3	Team 1: 1.1	Team 2: -1.1
Episode 391:	Steps: 600	Team 1 Lead: 14.3	Team 1: 0.0	Team 2: 0.0
Epi

Episode 501:	Steps: 161	Team 1 Lead: 20.9	Team 1: 1.1	Team 2: -1.1
Episode 502:	Steps: 600	Team 1 Lead: 20.9	Team 1: 0.0	Team 2: 0.0
Episode 503:	Steps: 48	Team 1 Lead: 19.8	Team 1: -1.1	Team 2: 1.1
Episode 504:	Steps: 600	Team 1 Lead: 19.8	Team 1: 0.0	Team 2: 0.0
Episode 505:	Steps: 77	Team 1 Lead: 20.9	Team 1: 1.1	Team 2: -1.1
Episode 506:	Steps: 600	Team 1 Lead: 20.9	Team 1: 0.0	Team 2: 0.0
Episode 507:	Steps: 600	Team 1 Lead: 20.9	Team 1: 0.0	Team 2: 0.0
Episode 508:	Steps: 600	Team 1 Lead: 20.9	Team 1: 0.0	Team 2: 0.0
Episode 509:	Steps: 600	Team 1 Lead: 20.9	Team 1: 0.0	Team 2: 0.0
Episode 510:	Steps: 48	Team 1 Lead: 22.0	Team 1: 1.1	Team 2: -1.1
Episode 511:	Steps: 169	Team 1 Lead: 20.9	Team 1: -1.1	Team 2: 1.1
Episode 512:	Steps: 87	Team 1 Lead: 19.8	Team 1: -1.1	Team 2: 1.1
Episode 513:	Steps: 248	Team 1 Lead: 18.7	Team 1: -1.1	Team 2: 1.1
Episode 514:	Steps: 490	Team 1 Lead: 17.6	Team 1: -1.1	Team 2: 1.1
Episode 515:	Steps: 600	Team 1 Lead: 17.6	Team 1: 0.0	Team 2: 0.0
Episod

...

Episode 5858:	Steps: 20	Team 1 Lead: 111.1	Team 1: 1.1	Team 2: -1.1
Episode 5859:	Steps: 600	Team 1 Lead: 111.1	Team 1: 0.0	Team 2: 0.0
Episode 5860:	Steps: 600	Team 1 Lead: 111.1	Team 1: 0.0	Team 2: 0.0
Episode 5861:	Steps: 600	Team 1 Lead: 111.1	Team 1: 0.0	Team 2: 0.0
Episode 5862:	Steps: 234	Team 1 Lead: 112.2	Team 1: 1.1	Team 2: -1.1
Episode 5863:	Steps: 600	Team 1 Lead: 112.2	Team 1: 0.0	Team 2: 0.0
Episode 5864:	Steps: 600	Team 1 Lead: 112.2	Team 1: 0.0	Team 2: 0.0
Episode 5865:	Steps: 600	Team 1 Lead: 112.2	Team 1: 0.0	Team 2: 0.0
Episode 5866:	Steps: 158	Team 1 Lead: 111.1	Team 1: -1.1	Team 2: 1.1
Episode 5867:	Steps: 227	Team 1 Lead: 112.2	Team 1: 1.1	Team 2: -1.1
Episode 5868:	Steps: 600	Team 1 Lead: 112.2	Team 1: 0.0	Team 2: 0.0
Episode 5869:	Steps: 600	Team 1 Lead: 112.2	Team 1: 0.0	Team 2: 0.0
Episode 5870:	Steps: 281	Team 1 Lead: 111.1	Team 1: -1.1	Team 2: 1.1
Episode 5871:	Steps: 600	Team 1 Lead: 111.1	Team 1: 0.0	Team 2: 0.0
Episode 5872:	Steps: 257	Team 1 Lead: 112.2	

Episode 5978:	Steps: 600	Team 1 Lead: 110.0	Team 1: 0.0	Team 2: 0.0
Episode 5979:	Steps: 288	Team 1 Lead: 111.1	Team 1: 1.1	Team 2: -1.1
Episode 5980:	Steps: 205	Team 1 Lead: 110.0	Team 1: -1.1	Team 2: 1.1
Episode 5981:	Steps: 214	Team 1 Lead: 111.1	Team 1: 1.1	Team 2: -1.1
Episode 5982:	Steps: 314	Team 1 Lead: 110.0	Team 1: -1.1	Team 2: 1.1
Episode 5983:	Steps: 600	Team 1 Lead: 110.0	Team 1: 0.0	Team 2: 0.0
Episode 5984:	Steps: 456	Team 1 Lead: 108.9	Team 1: -1.1	Team 2: 1.1
Episode 5985:	Steps: 178	Team 1 Lead: 110.0	Team 1: 1.1	Team 2: -1.1
Episode 5986:	Steps: 592	Team 1 Lead: 111.1	Team 1: 1.1	Team 2: -1.1
Episode 5987:	Steps: 201	Team 1 Lead: 110.0	Team 1: -1.1	Team 2: 1.1
Episode 5988:	Steps: 600	Team 1 Lead: 110.0	Team 1: 0.0	Team 2: 0.0
Episode 5989:	Steps: 600	Team 1 Lead: 110.0	Team 1: 0.0	Team 2: 0.0
Episode 5990:	Steps: 403	Team 1 Lead: 111.1	Team 1: 1.1	Team 2: -1.1
Episode 5991:	Steps: 472	Team 1 Lead: 110.0	Team 1: -1.1	Team 2: 1.1
Episode 5992:	Steps: 49	Team 1 Lead: 1

Episode 6098:	Steps: 600	Team 1 Lead: 114.4	Team 1: 0.0	Team 2: 0.0
Episode 6099:	Steps: 600	Team 1 Lead: 114.4	Team 1: 0.0	Team 2: 0.0
Episode 6100:	Steps: 79	Team 1 Lead: 115.5	Team 1: 1.1	Team 2: -1.1
Episode 6101:	Steps: 206	Team 1 Lead: 116.6	Team 1: 1.1	Team 2: -1.1
Episode 6102:	Steps: 600	Team 1 Lead: 116.6	Team 1: 0.0	Team 2: 0.0
Episode 6103:	Steps: 515	Team 1 Lead: 115.5	Team 1: -1.1	Team 2: 1.1
Episode 6104:	Steps: 600	Team 1 Lead: 115.5	Team 1: 0.0	Team 2: 0.0
Episode 6105:	Steps: 600	Team 1 Lead: 115.5	Team 1: 0.0	Team 2: 0.0
Episode 6106:	Steps: 105	Team 1 Lead: 114.4	Team 1: -1.1	Team 2: 1.1
Episode 6107:	Steps: 73	Team 1 Lead: 115.5	Team 1: 1.1	Team 2: -1.1
Episode 6108:	Steps: 484	Team 1 Lead: 114.4	Team 1: -1.1	Team 2: 1.1
Episode 6109:	Steps: 441	Team 1 Lead: 113.3	Team 1: -1.1	Team 2: 1.1
Episode 6110:	Steps: 600	Team 1 Lead: 113.3	Team 1: 0.0	Team 2: 0.0
Episode 6111:	Steps: 600	Team 1 Lead: 113.3	Team 1: 0.0	Team 2: 0.0
Episode 6112:	Steps: 416	Team 1 Lead: 114.4

Episode 6218:	Steps: 600	Team 1 Lead: 122.1	Team 1: 0.0	Team 2: 0.0
Episode 6219:	Steps: 600	Team 1 Lead: 122.1	Team 1: 0.0	Team 2: 0.0
Episode 6220:	Steps: 208	Team 1 Lead: 121.0	Team 1: -1.1	Team 2: 1.1
Episode 6221:	Steps: 50	Team 1 Lead: 122.1	Team 1: 1.1	Team 2: -1.1
Episode 6222:	Steps: 517	Team 1 Lead: 121.0	Team 1: -1.1	Team 2: 1.1
Episode 6223:	Steps: 569	Team 1 Lead: 122.1	Team 1: 1.1	Team 2: -1.1
Episode 6224:	Steps: 251	Team 1 Lead: 121.0	Team 1: -1.1	Team 2: 1.1
Episode 6225:	Steps: 78	Team 1 Lead: 122.1	Team 1: 1.1	Team 2: -1.1
Episode 6226:	Steps: 588	Team 1 Lead: 121.0	Team 1: -1.1	Team 2: 1.1
Episode 6227:	Steps: 600	Team 1 Lead: 121.0	Team 1: 0.0	Team 2: 0.0
Episode 6228:	Steps: 600	Team 1 Lead: 121.0	Team 1: 0.0	Team 2: 0.0
Episode 6229:	Steps: 258	Team 1 Lead: 119.9	Team 1: -1.1	Team 2: 1.1
Episode 6230:	Steps: 600	Team 1 Lead: 119.9	Team 1: 0.0	Team 2: 0.0
Episode 6231:	Steps: 600	Team 1 Lead: 119.9	Team 1: 0.0	Team 2: 0.0
Episode 6232:	Steps: 600	Team 1 Lead: 119.

Episode 6338:	Steps: 600	Team 1 Lead: 127.6	Team 1: 0.0	Team 2: 0.0
Episode 6339:	Steps: 600	Team 1 Lead: 127.6	Team 1: 0.0	Team 2: 0.0
Episode 6340:	Steps: 315	Team 1 Lead: 128.7	Team 1: 1.1	Team 2: -1.1
Episode 6341:	Steps: 142	Team 1 Lead: 127.6	Team 1: -1.1	Team 2: 1.1
Episode 6342:	Steps: 565	Team 1 Lead: 126.5	Team 1: -1.1	Team 2: 1.1
Episode 6343:	Steps: 600	Team 1 Lead: 126.5	Team 1: 0.0	Team 2: 0.0
Episode 6344:	Steps: 216	Team 1 Lead: 127.6	Team 1: 1.1	Team 2: -1.1
Episode 6345:	Steps: 31	Team 1 Lead: 126.5	Team 1: -1.1	Team 2: 1.1
Episode 6346:	Steps: 600	Team 1 Lead: 126.5	Team 1: 0.0	Team 2: 0.0
Episode 6347:	Steps: 283	Team 1 Lead: 125.4	Team 1: -1.1	Team 2: 1.1
Episode 6348:	Steps: 247	Team 1 Lead: 126.5	Team 1: 1.1	Team 2: -1.1
Episode 6349:	Steps: 600	Team 1 Lead: 126.5	Team 1: 0.0	Team 2: 0.0
Episode 6350:	Steps: 193	Team 1 Lead: 127.6	Team 1: 1.1	Team 2: -1.1


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(team_0_lead)+1), team_0_lead)
plt.ylabel('Team #1 Cumulative Lead')
plt.xlabel('Episode #')
plt.show()

### Watch The Trained Agents

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

striker_0 = Policy(s_state_size, s_action_size, hidden_0, hidden_1, hidden_2).to(device)
goalie_0 = Policy(g_state_size, g_action_size, hidden_0, hidden_1, hidden_2).to(device)

# Load the saved training parameters
striker_0.load_state_dict(torch.load('checkpoint_striker_0.pth', map_location='cpu'))
goalie_0.load_state_dict(torch.load('checkpoint_goalie_0.pth', map_location='cpu'))

# Set environment to evalulation mode
env_info = env.reset(train_mode=False)        
g_states = env_info[g_brain_name].vector_observations  
s_states = env_info[s_brain_name].vector_observations  

for i in range(20):
    while True:
        states_all = combine_states(g_states, s_states)
        g0_prob, g0_action = goalie_0(states_all)
        g1_action = np.random.randint(g_action_size)
        g_actions = np.array([g0_action, g1_action])
        s0_prob, s0_action = striker_0(states_all)
        s1_action = np.random.randint(s_action_size)
        s_actions = np.array([s0_action, s1_action])
        actions = dict(zip([g_brain_name, s_brain_name], [g_actions, s_actions]))
        env_info = env.step(actions)                       

        g_next_states = env_info[g_brain_name].vector_observations         
        s_next_states = env_info[s_brain_name].vector_observations

        done = np.any(env_info[g_brain_name].local_done)  

        # exit loop if episode finished
        if done:                                           
            break

        # roll over states to next time step
        g_states = g_next_states
        s_states = s_next_states

When finished, you can close the environment.

In [ ]:
#env.close()